In [31]:
from keras.datasets import cifar10
import numpy as np
from keras.utils.np_utils import to_categorical
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn as nn
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import keras
from keras import layers
from skimage.feature import hog
import cv2
from PIL import Image
import time

In [32]:
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim

dataroot = 'bmo_mouse/'
dataset = dset.ImageFolder(root=dataroot,
                               transform=transforms.Compose([
                                   transforms.Resize(64),
                                   transforms.CenterCrop(64),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ]))

print(len(dataset.targets))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64,
                                             shuffle=True, num_workers=0)

1065


In [33]:
transform=transforms.Compose([
                                   transforms.Resize(64),
                                   transforms.CenterCrop(64),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               ])

In [34]:
for i,data in enumerate(dataloader,0):
    if i ==0:
        x_train = feature_net(data[0])
        y_train = data[1]
    else:
        x_train = torch.cat((x_train,feature_net(data[0])))
        y_train = torch.cat((y_train,data[1]))
x_train2 = np.array(x_train.detach())
y_train2 = np.array(y_train.detach())
x_train2 = x_train2.reshape((1065,512*4*4))
y_train2 = y_train2

In [35]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(x_train2,y_train2)
dataset.class_to_idx

{'bmo': 0, 'mouse': 1}

In [37]:
def square_box(img):
    start = time.time()
    finalx=0
    x=0
    y=0
    count = 0
    for i in range(0,720,50):
        if i + 360 < 720:
            img2=img[:,i:i+360,:]
            pil_img = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
            img3 = transform(pil_img).reshape((1,3,64,64))
            feature = feature_net(img3)
            feature = np.array(feature.detach())
            feature = feature.reshape((1,512*4*4))
            if clf.predict(feature)==1:
                count = count+1
                img2=img[:,i+100:i+350,:]
                pil_img = Image.fromarray(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
                img3 = transform(pil_img).reshape((1,3,64,64))
                feature2 = feature_net(img3)
                feature2 = np.array(feature2.detach())
                feature2 = feature2.reshape((1,512*4*4))
                if clf.predict(feature2)==1:
                    x = i+100
                   # print(x)
                    img = cv2.rectangle(img,pt1=(x,0),pt2=(x+250,404),color=(255,255,0),thickness=2,lineType=cv2.LINE_AA)     
    print(time.time()-start)
    return x, img

In [38]:
def square_box2(img,x):
    start = time.time()
    for i in range(0,min(720-x,x),20):
        img2=img[:,x:x+360,:]
        img3=img[:,x+i:x+i+360,:]
        img4=img[:,x-i:x-i+360,:]
        feature2 = img_process(img2)
        feature3 = img_process(img3)
        feature4 = img_process(img4)
        newx=x
        if clf.predict(feature2)==1:
            newx = x
            img = cv2.rectangle(img,pt1=(newx,0),pt2=(newx+250,404),color=(255,255,0),thickness=2,lineType=cv2.LINE_AA)
            break
        else:
            if clf.predict(feature3)==1:
                newx = x + i
                img = cv2.rectangle(img,pt1=(newx,0),pt2=(newx+250,404),color=(255,255,0),thickness=2,lineType=cv2.LINE_AA)            
                break
            else:
                if clf.predict(feature4)==1:
                    newx = x-i
                    img = cv2.rectangle(img,pt1=(newx,0),pt2=(newx+250,404),color=(255,255,0),thickness=2,lineType=cv2.LINE_AA)    
                    break
        #print(newx)
        print(time.time()-start)
    return newx, img

In [39]:
def img_process(img):
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img = transform(pil_img).reshape((1,3,64,64))
    feature = feature_net(img)
    feature = np.array(feature.detach())
    feature = feature.reshape((1,512*4*4))
    return feature

In [42]:
cap = cv2.VideoCapture('123.mp4')
ret,img = cap.read()

while True:
    ret,img = cap.read()
    if not ret:
        break
    x,img = square_box(img)
    cv2.imshow('a',img)
    key = cv2.waitKey(1)
    if key==27:
        cv2.destroyAllWindows()
        break

0.14499592781066895
0.16699767112731934
0.14899897575378418
0.13899922370910645
0.13099956512451172
0.13699913024902344
0.13200044631958008
0.15299725532531738
0.13801026344299316
0.15199971199035645
0.13699865341186523
0.14299964904785156
0.1509993076324463
0.14699912071228027
0.1439990997314453
0.17599940299987793
0.16100049018859863
0.15999984741210938
0.14099812507629395
0.14299941062927246
0.15400052070617676
0.16698765754699707
0.1547083854675293
0.1470015048980713
0.13700056076049805
0.16099953651428223
0.16103482246398926
0.15599846839904785
0.1409926414489746
0.15899968147277832
0.15200138092041016
0.14300036430358887
0.15199732780456543
0.14799761772155762
0.14399981498718262
0.16352105140686035
0.12799811363220215
0.1329967975616455
0.13199901580810547
0.14799761772155762
0.13899755477905273
0.13199710845947266
0.13599872589111328
0.13199830055236816
0.13151121139526367
0.12600088119506836
0.11900663375854492
0.1509995460510254
0.25774383544921875
0.13499784469604492
0.13899

In [44]:
cap = cv2.VideoCapture('123.mp4')
ret,img = cap.read()
count = 0
while True:
    ret,img = cap.read()
    if not ret:
        break
    if count == 0:
        x,img = square_box(img)
        count=count+1
    else: 
        x,img = square_box2(img,x)
    cv2.imshow('a',img)
    key = cv2.waitKey(1)
    if key==27:
        cv2.destroyAllWindows()
        break

0.1309986114501953
0.048998355865478516
0.04500007629394531
0.04199504852294922
0.048508405685424805
0.04000115394592285
0.05409121513366699
0.042999267578125
0.05304217338562012
0.043001413345336914
0.045000553131103516
0.03899669647216797
0.04200029373168945
0.03899955749511719
0.040001630783081055
0.04199838638305664
0.041002511978149414
0.04200267791748047
0.04099750518798828
0.04799938201904297
0.041001081466674805
0.04900169372558594
0.04000353813171387
0.05200695991516113
0.041999101638793945
0.052000999450683594
0.05099964141845703
0.04199934005737305
